<a href="https://colab.research.google.com/github/paty-oliveira/stock-markets-analytics-zoomcamp/blob/main/Homework_Assignment_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
import numpy as np
import re
import yfinance as yf
import numpy as np


In [2]:
# Auxiliar functions

def calculate_price_range(input):
  pattern = r'\$(\d+\.\d+) - \$(\d+\.\d+)'

  match = re.match(pattern, input)

  if match:
    first_value = float(match.group(1))
    second_value = float(match.group(2))
    return (first_value + second_value) / 2
  else:
    return float(input.replace("$", ""))


**Question 1:** What's the total sum ($m) of 2023 filings that happenned of Fridays?



In [13]:
endpoint = "https://stockanalysis.com/ipos/filings/"
response = requests.get(endpoint)

df = pd.read_html(response.text, na_values=["-"])
ipos = df[0]
ipos.head()


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,NaN,NaN
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,NaN,NaN


In [ ]:
ipos["Filing Date"] = pd.to_datetime(ipos["Filing Date"])
ipos["Shares Offered"] = pd.to_numeric(ipos["Shares Offered"])
ipos["Average Price"] = ipos["Price Range"].astype(str).apply(lambda x:calculate_price_range(x))
ipos["Shared Offered Value"] = ipos["Shares Offered"] * ipos["Average Price"]
ipos.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Average Price,Shared Offered Value
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
3,2024-04-26,HDL,Super Hi International Holding Ltd.,NaN,NaN,NaN,NaN
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.0,NaN


In [ ]:
year = pd.DatetimeIndex(ipos["Filing Date"]).year
day_of_week = pd.DatetimeIndex(ipos["Filing Date"]).dayofweek

ipos_2023 = ipos[(year == 2023) & (day_of_week == 4)]
print("Number of Rows: ", ipos_2023.shape[0])

sum = round(ipos_2023["Shared Offered Value"].sum() / 1e6)
print("Total of Milions: ", sum, "$M")

Number of Rows:  32
Total of Milions:  286 $M


**Question 2:** Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

In [ ]:
endpoint_ipos_2023 = "https://stockanalysis.com/ipos/2023/"
endpoint_ipos_2024 = "https://stockanalysis.com/ipos/2024/"

response_ipos_2023 = requests.get(endpoint_ipos_2023)
response_ipos_2024 = requests.get(endpoint_ipos_2024)

df_ipos_2023 = pd.read_html(response_ipos_2023.text)[0]
df_ipos_2024 = pd.read_html(response_ipos_2024.text)[0]

ipos = pd.concat([df_ipos_2023, df_ipos_2024], ignore_index=True)

ipos.head()


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.42%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$3.11,-22.15%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$11.96,-22.84%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.17,1.70%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.76,-80.98%


In [ ]:
ipos["IPO Date"] = pd.to_datetime(ipos["IPO Date"])
ipos["IPO Price"] = pd.to_numeric(ipos["IPO Price"].astype("str").str.replace("$", ""), errors="coerce")
ipos["Current"] = pd.to_numeric(ipos["Current"].str.replace("$", ""), errors="coerce")
ipos["Return"] = pd.to_numeric(ipos["Return"].str.replace("%", ""), errors="coerce")
ipos = ipos[ipos['Symbol'] != 'RYZB']


# filter data
ipos.info()


<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      217 non-null    datetime64[ns]
 1   Symbol        217 non-null    object        
 2   Company Name  217 non-null    object        
 3   IPO Price     217 non-null    float64       
 4   Current       217 non-null    float64       
 5   Return        216 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 11.9+ KB


In [ ]:

def calculate_growth(df, day):
  return df['Adj Close'] / df['Adj Close'].shift(day)

tickers = ipos["Symbol"].unique()

df = None

for ticker in tickers:
  df = yf.download(tickers=ticker, period="max", interval="1d")

  for day in range(1, 31):
    df["growth_future_" + str(day) + "d"] = calculate_growth(df, day)

stats = df.iloc[:, 6:].describe()
stats.reset_index(inplace=True)
percentile = stats.iloc[6]
percentile

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

index                     75%
growth_future_1d     1.029782
growth_future_2d         1.06
growth_future_3d     1.070713
growth_future_4d     1.060052
growth_future_5d     1.069493
growth_future_6d        1.032
growth_future_7d     1.034345
growth_future_8d     1.029783
growth_future_9d     1.029982
growth_future_10d    1.024793
growth_future_11d    1.005906
growth_future_12d    1.004055
growth_future_13d    1.068501
growth_future_14d    1.105769
growth_future_15d    1.090278
growth_future_16d    1.085481
growth_future_17d    1.102083
growth_future_18d    1.144444
growth_future_19d     1.12037
growth_future_20d    1.144294
growth_future_21d    1.206209
growth_future_22d    1.126214
growth_future_23d    1.143689
growth_future_24d    1.181731
growth_future_25d    1.259899
growth_future_26d    1.085271
growth_future_27d    1.124585
growth_future_28d    1.094407
growth_future_29d    1.170721
growth_future_30d       1.125
Name: 6, dtype: object

**Question 3:** Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?



In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

ALL_STOCKS = LARGEST_STOCKS + LARGE_STOCKS

start = "2014-01-01"
end = "2023-12-31"

stocks_df = pd.DataFrame()

for ticker in ALL_STOCKS:

  historic_prices = yf.download(tickers=ticker, start=start, end=end, interval="1d")
  historic_prices["Ticker"] = ticker
  historic_prices["growth_7d"] = historic_prices["Adj Close"] / historic_prices["Adj Close"].shift(7)
  stocks_df = pd.concat([stocks_df, historic_prices])

In [27]:
categories = {
    "LARGEST": LARGEST_STOCKS,
    "LARGE": LARGE_STOCKS
}

def get_ticker_category(ticker):
  return next((category for category, stocks in categories.items() if ticker in stocks), "ERROR")

In [71]:
stocks_df["ticker_category"] = stocks_df["Ticker"].apply(lambda ticker: get_ticker_category(ticker))

avg_growth_by_category = stocks_df.pivot_table(index='Date', columns='ticker_category', values='growth_7d', aggfunc='mean')
avg_growth_by_category["outperform"] = (avg_growth_by_category["LARGE"] > avg_growth_by_category["LARGEST"])
avg_growth_by_category.head()

ticker_category,LARGE,LARGEST,outperform
Date,,,
2014-01-10,0.976317,0.967215,True
2014-01-13,0.998538,0.992337,True
2014-01-14,1.008288,0.998351,True
2014-01-15,1.015493,1.011325,True
2014-01-16,1.013890,1.006760,True


In [73]:
outperform_days = avg_growth_by_category["outperform"].sum()
total_days = avg_growth_by_category.shape[0]

percentage_outperform = round((outperform_days / total_days) * 100)
print("Percentage of days outperforming:", percentage_outperform, "%")

Percentage of days outperforming: 47 %


**Question 4:** What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

In [17]:
path = "/content/stocks_df_combined_trunc_2014_2023.parquet.brotli"

df = pd.read_parquet(path)
df.tail()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301
5342,3531.000000,3540.000000,3495.000000,3526.000000,3526.000000,968577.0,LT.NS,2023,2023-12-01,4,...,0.972359,0.995092,0.925850,0.720539,0.987603,0.990099,0.956850,1.112020,1.561092,2.529656


In [54]:
df = df[df["Weekday"] == 4]
df = df[df["cci"] > 200]
df.tail()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,profit_loss,pips,diff
7015,437.899994,450.950012,437.399994,449.799988,443.289459,19336856.0,ITC.NS,2023,2023-12-01,4,...,0.705924,1.025880,1.022671,1.025675,1.091754,1.495576,2.280217,12.307525,0.012308,0.012308
4714,642.900024,667.000000,640.000000,663.750000,571.743835,1747468.0,HINDUNILVR.NS,2014,2014-07-01,4,...,0.955146,NaN,NaN,NaN,NaN,NaN,NaN,-110.680022,-0.110680,-0.110680
4823,818.500000,868.400024,817.049988,864.599976,750.911133,3825060.0,HINDUNILVR.NS,2015,2015-01-01,4,...,0.458211,1.024913,1.014742,0.921836,0.838445,0.801570,NaN,-82.576502,-0.082577,-0.082577
3133,1055.000000,1074.666626,1040.000000,1061.166626,922.283386,4195027.0,LT.NS,2015,2015-01-01,4,...,0.443010,0.991675,0.921350,0.716568,0.650759,0.531616,NaN,-125.797738,-0.125798,-0.125798
3466,982.666687,991.566650,968.000000,983.266663,863.330994,5885740.0,LT.NS,2016,2016-05-01,4,...,0.807730,1.044289,1.061624,1.069550,1.064713,1.094666,1.994305,-121.440662,-0.121441,-0.121441


In [56]:
initial_investment = 1000

def calculate_gross_profit(open_price, close_price):
  return((close_price - open_price) / open_price)


df["diff"] = calculate_gross_profit(df["Open"], df["Adj Close_x"])
df["profit_loss"] = df["diff"] * initial_investment
sum_profit_loss = round(df["profit_loss"].sum())

print("Profit Loss: ", sum_profit_loss)

Profit Loss:  -24830


**Question 5:** Finding Your Strategy for IPOs

In [26]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}
endpoint = "https://stockanalysis.com/api/screener/i/bd/revenue+debt+liabilities+assets+grossMargin+netIncome.json"

request = requests.get(endpoint, headers=headers)
response = request.json()["data"]["data"]

df = pd.DataFrame.from_dict(response, orient="columns")
enriched_df = df.transpose()
enriched_df.head()

,revenue,debt,grossMargin,netIncome,liabilities,assets
ZK,6.089569e+09,2.436356e+09,8.68564,-1.189166e+09,NaN,NaN
IFIT,1.745056e+09,NaN,NaN,-5.167060e+08,NaN,NaN
PHCC,1.501847e+08,NaN,NaN,1.147572e+08,NaN,NaN
HCG,1.346783e+08,NaN,NaN,-1.903605e+07,NaN,NaN
GGL,3.968061e+08,6.163577e+06,56.01635,1.122262e+08,NaN,NaN


My strategy to find potential good investments in IPOs:

- Gathering additional information about each company, such as revenue, debt, liabilities, assets, gross margin and net income.
- Filter data based on the following statements:
  * Companies with net income greater than 0
  * Companies with assets greater than 0
  * Companies with more assets than debt
  * Companies with gross margin greater than 70
- After this first triage, I would analyse each company carefully in terms of fundamentals: Income, Balanche Sheet, Cash Flow and most important Ratios.
- After that, I would compare those against existing public traded stocls and verify if the companies are following the average of the sector/industry.


In [27]:
enriched_df.info()
enriched_df.fillna(0)

<class 'pandas.core.frame.DataFrame'>
Index: 259 entries, ZK to LMIN
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   revenue      223 non-null    float64
 1   debt         199 non-null    float64
 2   grossMargin  180 non-null    float64
 3   netIncome    253 non-null    float64
 4   liabilities  63 non-null     float64
 5   assets       63 non-null     float64
dtypes: float64(6)
memory usage: 14.2+ KB


,revenue,debt,grossMargin,netIncome,liabilities,assets
ZK,6.089569e+09,2.436356e+09,8.68564,-1.189166e+09,0.0,0.0
IFIT,1.745056e+09,0.000000e+00,0.00000,-5.167060e+08,0.0,0.0
PHCC,1.501847e+08,0.000000e+00,0.00000,1.147572e+08,0.0,0.0
HCG,1.346783e+08,0.000000e+00,0.00000,-1.903605e+07,0.0,0.0
GGL,3.968061e+08,6.163577e+06,56.01635,1.122262e+08,0.0,0.0
...,...,...,...,...,...,...
ACSB,0.000000e+00,0.000000e+00,0.00000,-1.546282e+06,0.0,0.0
LEC,0.000000e+00,0.000000e+00,0.00000,-4.311020e+05,0.0,0.0
FP,0.000000e+00,0.000000e+00,0.00000,-2.921325e+06,0.0,0.0
RISE,0.000000e+00,0.000000e+00,0.00000,-2.172193e+06,0.0,0.0


In [28]:
enriched_df = enriched_df[enriched_df["netIncome"] > 0] # Remove companies with netIncome < 0
enriched_df = enriched_df[enriched_df["assets"] > 0] # Remove companies with assets < 0 or without information about it
enriched_df = enriched_df[enriched_df["assets"] > enriched_df["debt"]] # Remove companies with more debt than assets
enriched_df = enriched_df[enriched_df["grossMargin"] > 70] # Remove companies with Gross margin < 70
enriched_df.head(7)

,revenue,debt,grossMargin,netIncome,liabilities,assets
GLXG,5.978576e+06,6.446652e+05,75.40018,2874701.0,2.247305e+06,4.192977e+06
EEET,1.671351e+06,5.518670e+05,76.05841,996157.0,8.128660e+05,2.009318e+06
ZRSP,1.359185e+07,1.811042e+07,97.17672,215520.0,2.067319e+07,2.155960e+07
BOUW,1.180073e+06,2.157830e+05,100.00000,636882.0,3.434350e+05,1.296650e+06
QQJ,1.040796e+07,7.604100e+04,76.82963,2050719.0,1.271244e+07,2.139415e+07
BKV,1.553881e+09,7.687040e+08,100.00000,359277000.0,1.811085e+09,2.748365e+09
ALEF,1.311390e+08,3.214800e+07,87.43318,20975000.0,3.368390e+08,3.922540e+08
